In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
PATH_OF_DATA= '/content/gdrive/"My Drive"/foreground'
!ls {PATH_OF_DATA}

In [ ]:
import os
os.listdir('/content/gdrive/My Drive/foreground')

In [ ]:
import shutil
catoonized_images = 'cartooonized_images'
if os.path.exists(catoonized_images):
  shutil.rmtree(catoonized_images)
os.makedirs(catoonized_images)

In [ ]:
catoonized_images = 'test_images'
if os.path.exists(catoonized_images):
  shutil.rmtree(catoonized_images)
os.makedirs(catoonized_images)

In [ ]:
catoonized_images = 'saved_models'
if os.path.exists(catoonized_images):
  shutil.rmtree(catoonized_images)
os.makedirs(catoonized_images)

In [ ]:
import os
os.listdir()

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
os.listdir()

In [ ]:
os.chdir('test_images/')

In [ ]:
import shutil
import os
lists=os.listdir('/content/gdrive/My Drive/foreground')
for lis in lists:
  shutil.move(f'/content/gdrive/My Drive/foreground/{lis}','/content/test_images')

In [ ]:
os.chdir('../')

In [ ]:
os.chdir('saved_models/')

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
os.chdir('../')

In [ ]:
os.getcwd()

'/content'

In [ ]:
pip install network

In [ ]:
pip install tensorflow==1.13.0rc1

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf 
import network
import guided_filter
from tqdm import tqdm

In [ ]:
def resize_crop(image):
    h, w, c = np.shape(image)
    if min(h, w) > 720:
        if h > w:
            h, w = int(720*h/w), 720
        else:
            h, w = 720, int(720*w/h)
    image = cv2.resize(image, (w, h),
                       interpolation=cv2.INTER_AREA)
    h, w = (h//8)*8, (w//8)*8
    image = image[:h, :w, :]
    return image
    

In [ ]:
def cartoonize(load_folder, save_folder, model_path):
    input_photo = tf.placeholder(tf.float32, [1, None, None, 3])
    network_out = network.unet_generator(input_photo)
    final_out = guided_filter.guided_filter(input_photo, network_out, r=1, eps=5e-3)

    all_vars = tf.trainable_variables()
    gene_vars = [var for var in all_vars if 'generator' in var.name]
    saver = tf.train.Saver(var_list=gene_vars)
  
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)

    sess.run(tf.global_variables_initializer())
    saver.restore(sess, tf.train.latest_checkpoint(model_path))
    name_list = os.listdir(load_folder)
    for name in tqdm(name_list):
        try:
            load_path = os.path.join(load_folder, name)
            save_path = os.path.join(save_folder, name)
            image = cv2.imread(load_path)
            image = resize_crop(image)
            batch_image = image.astype(np.float32)/127.5 - 1
            batch_image = np.expand_dims(batch_image, axis=0)
            output = sess.run(final_out, feed_dict={input_photo: batch_image})
            output = (np.squeeze(output)+1)*127.5
            output = np.clip(output, 0, 255).astype(np.uint8)
            cv2.imwrite(save_path, output)
        except:
            print('cartoonize {} failed'.format(load_path))

In [ ]:
if __name__ == '__main__':
    model_path = 'saved_models'
    load_folder = 'test_images'
    save_folder = 'cartooonized_images'
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
    cartoonize(load_folder, save_folder, model_path)

In [ ]:
os.listdir()

In [ ]:
os.chdir('cartooonized_images/')

In [ ]:
os.listdir()

In [ ]:
os.chdir('../')

In [ ]:
os.listdir()

In [ ]:
if os.path.exists("/content/drive/My Drive/cartooonized_images"):
  shutil.rmtree("/content/drive/My Drive/cartooonized_images")
!cp  -r 'cartooonized_images/' "gdrive/My Drive/"